In [ ]:
import os
import sys 
os.chdir('..')
sys.path.insert(0, os.getcwd())
import numpy as np
import torch
from torch.utils.data import DataLoader

import glob
from tfrecord.tools import index_tfrecord

from Data_loader import load_subjects_from_json, get_all_npy_paths_by_group, base_folders

#from Dimensionality_Reduction import plot_embedding_3d



In [2]:
from AE_pipeline_pytorch import (
    LSTMAutoencoder,
    BiLSTMAutoencoder,
    create_dataloader,
    train_autoencoder,
    evaluate_and_detect,
    extract_and_save_latents,
    reconstruct_and_evaluate 
)

2025-06-25 19:01:07.127286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750888867.144119   76355 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750888867.149162   76355 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750888867.162696   76355 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750888867.162716   76355 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750888867.162718   76355 computation_placer.cc:177] computation placer alr

Using device: cuda


In [ ]:
from tfrecord_utils import create_tfrecord_dataloader

In [4]:
BATCH_SIZE = 256
N_TIMESTEPS = 100
N_VARS      = 321
LATENT_DIM  = 128   
EPOCHS      = 30   

In [ ]:
#Generate the index for the TFRecord files
for folder in ["data/train_tfrecords", "data/val_tfrecords", "data/test_tfrecords"]:
    for tfp in sorted(glob.glob(f"{folder}/*.tfrecord.gz")):
        idxp = tfp.replace(".tfrecord.gz", ".idx")
        index_tfrecord(tfp, idxp)
        print(f"Índice creado: {idxp}")

In [ ]:
# Create DataLoader for TFRecord files

def get_paths(folder: str):
    tf_paths = sorted(glob.glob(os.path.join(folder, "*.tfrecord.gz")))
    idx_paths = [p.replace(".tfrecord.gz", ".idx") for p in tf_paths]
    return tf_paths, idx_paths

train_tf, train_idx = get_paths("data/train_tfrecords")
val_tf,   val_idx   = get_paths("data/val_tfrecords")
test_tf,  test_idx  = get_paths("data/test_tfrecords")

train_loader = create_tfrecord_dataloader(
    tfrecord_paths=train_tf,
    idx_paths=train_idx,
    batch_size=BATCH_SIZE,
    shuffle_queue=1024,
    num_workers=4,
    pin_memory=True
)
val_loader = create_tfrecord_dataloader(
    tfrecord_paths=val_tf,
    idx_paths=val_idx,
    batch_size=BATCH_SIZE,
    shuffle_queue=0,
    num_workers=2,
    pin_memory=True
)
test_loader = create_tfrecord_dataloader(
    tfrecord_paths=test_tf,
    idx_paths=test_idx,
    batch_size=BATCH_SIZE,
    shuffle_queue=0,
    num_workers=2,
    pin_memory=True
)


In [7]:
#Generar SHARDS para TRAIN 
shards_dir = "train_shards"
if not os.path.isdir(shards_dir):
    write_sharded_tfrecord(
        npy_paths=train_npy,
        output_dir=shards_dir,
        shard_size=5_000
    )
    print(f"→ Shards generados en: {shards_dir}/")
else:
    print(f"→ Shards ya existen en: {shards_dir}/")

→ Shards ya existen en: train_shards/


In [8]:
#Convertir VAL y TEST a TFRecord monolítico 
for split, npy_list in [("val", val_npy), ("test", test_npy)]:
    tfp = f"{split}_cycles.tfrecord.gz"
    if not os.path.exists(tfp):
        convert_npy_to_tfrecord(npy_list, tfp)
        print(f"Converted → {tfp}")
    else:
        print(f"Skipping (already exists) → {tfp}")

Skipping (already exists) → val_cycles.tfrecord.gz
Skipping (already exists) → test_cycles.tfrecord.gz


In [ ]:
#Ya no es necesario Create tf.data.Dataset
# 3a) Lista de archivos shard
shard_files = sorted(glob.glob(os.path.join(shards_dir, "*.tfrecord.gz")))

# 3b) Pipeline shard-aware
train_ds = (
   tf.data.Dataset
      .list_files(shard_files, shuffle=True)
      .interleave(
         lambda f: tf.data.TFRecordDataset(f, compression_type="GZIP"),
         cycle_length=4,
         num_parallel_calls=tf.data.AUTOTUNE
      )
      .map(_parse_cycle, num_parallel_calls=tf.data.AUTOTUNE)
      #.repeat() # Evita out of range 
      .shuffle(5_000, seed=42)
      .batch(BATCH_SIZE, drop_remainder=True)
      .prefetch(tf.data.AUTOTUNE)
)

# VAL y TEST: dataset monolítico


val_ds  = make_monolithic_ds("val_cycles.tfrecord.gz")
test_ds = make_monolithic_ds("test_cycles.tfrecord.gz")

print(f"→ train_ds: {train_ds}")
print(f"→ val_ds:   {val_ds}")
print(f"→ test_ds:  {test_ds}")


I0000 00:00:1750879970.180979   90022 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4977 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


→ train_ds: <_PrefetchDataset element_spec=(TensorSpec(shape=(256, 100, 321), dtype=tf.float32, name=None), TensorSpec(shape=(256, 100, 321), dtype=tf.float32, name=None))>
→ val_ds:   <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None))>
→ test_ds:  <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None))>


In [ ]:
# Optional from a batch validate if still NaN 
for x_batch, y_batch in train_ds.take(1):
      import tensorflow as tf
      print("Input  ◂ min:", tf.reduce_min(x_batch).numpy(),
            "max:", tf.reduce_max(x_batch).numpy(),
            "mean:", tf.reduce_mean(x_batch).numpy(),
            "std:", tf.math.reduce_std(x_batch).numpy())
      print("Target ◂", 
            tf.reduce_min(y_batch).numpy(), tf.reduce_max(y_batch).numpy())
      # Comprueba si hay NaN/Inf
      print("Any NaN in x?", tf.reduce_any(tf.math.is_nan(x_batch)).numpy())
      print("Any Inf in x?", tf.reduce_any(tf.math.is_inf(x_batch)).numpy())
      break


Input  ◂ min: -12.165039 max: 15.267122 mean: -0.009751429 std: 1.0221982
Target ◂ -12.165039 15.267122
Any NaN in x? False
Any Inf in x? False


In [10]:
#Optional Identify # cycles in train and validation 
import numpy as np, math
# 1) Total de ciclos en train/val
total_train_cycles = sum(np.load(p).shape[0] for p in train_npy)
total_val_cycles   = sum(np.load(p).shape[0] for p in val_npy)
print(total_train_cycles)
print(total_val_cycles)

194774
42317


In [7]:
#Optional steps_per_epoch & validation_steps
total_train_cycles=194774
total_val_cycles=42317
steps_per_epoch    = total_train_cycles // BATCH_SIZE
validation_steps   = total_val_cycles   // BATCH_SIZE
print(steps_per_epoch)
print(validation_steps)

1521
330


In [11]:
# 1) Generar lista de shards y de índices (*.idx)
tf_paths  = sorted(glob("data/train_tfrecords/*.tfrecord.gz"))
idx_paths = [p.replace(".tfrecord.gz", ".idx") for p in tf_paths]

# 2) Crear el DataLoader
train_loader = create_tfrecord_dataloader(
    tfrecord_paths=tf_paths,
    idx_paths=idx_paths,
    batch_size=256,
    shuffle=True
)

val_loader = create_dataloader(
    val_npy,
    batch_size=BATCH_SIZE,
    is_train=False,
    n_timesteps=N_TIMESTEPS,
    n_vars=N_VARS
)

test_loader = create_dataloader(
    test_npy,
    batch_size=BATCH_SIZE,
    is_train=False,
    n_timesteps=N_TIMESTEPS,
    n_vars=N_VARS
)

TypeError: 'module' object is not callable

In [ ]:
model = LSTMAutoencoder(n_timesteps=N_TIMESTEPS,
                        n_vars=N_VARS,
                        latent_dim=LATENT_DIM).to(device)

In [ ]:
train_autoencoder(
    model,
    train_loader,
    val_loader,
    run_id="run1",
    epochs=EPOCHS,
)

In [ ]:
losses, threshold = evaluate_and_detect(model, test_loader)

In [ ]:
latents = extract_and_save_latents(model, test_loader, output_path="latents_test.npy")
plot_embedding_3d(latents)

In [7]:
#Build and train the Autoencoder
# Hiperparameters 
def r2(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    ss_total = K.sum(K.square(y_true - K.mean(y_true)))
    ss_residual = K.sum(K.square(y_true - y_pred))
    return 1 - (ss_residual / ss_total)

run_id = "256_tanh_lr1e4_100ep_AdamW"
n_timesteps = 100
n_vars = 321
latent_dim = 256
epochs = 100
lr_initial   = 1e-4
lr_decay_rate = 0.98
lr_decay_steps = 5000
clipnorm     = 1.0
steps_per_epoch  = 1521 #batch 128
validation_steps = 330 #batch 128

model = build_lstm_autoencoder(
    n_timesteps=n_timesteps,
    n_vars=n_vars,
    latent_dim=latent_dim,
    )

history = train_autoencoder(
        model=model,
        train_ds=train_ds,
        val_ds=val_ds,
        run_id=run_id,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps
        )

        

Epoch 1/100
    814/Unknown 245s 290ms/step - loss: 1.6083 - r2: 0.2305 - root_mean_squared_error: 0.9269

KeyboardInterrupt: 